In [ ]:
import recpack

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv(
    'gs://froomle-playground-robin-1-data/datasets/ML_100K/u.data',
    sep='\t',
    header=None,
    names = ['userId', 'movieId', 'rating', 'timestamp'])

In [ ]:
from recpack import algorithms

In [ ]:
from recpack import helpers

In [ ]:
from recpack import evaluate

In [ ]:
item_ids = list(dataframe['movieId'].unique())
user_ids = list(dataframe['userId'].unique())

In [ ]:
item_id_mapping = helpers.rescale_id_space(item_ids)

In [ ]:
user_id_mapping = helpers.rescale_id_space(user_ids)

In [ ]:
dataframe['iid'] = dataframe['movieId'].map(lambda x: item_id_mapping[x])
dataframe['uid'] = dataframe['userId'].map(lambda x: user_id_mapping[x])

In [ ]:
df = dataframe.drop(['userId', 'movieId', 'rating', 'timestamp'], axis=1)

In [ ]:
# Still need to move over the train test split function. 

In [ ]:
sp_mat = helpers.create_csr_matrix_from_pandas_df(df, 'iid', 'uid')

In [ ]:
algo = algorithms.get_algorithm('SVD')

In [ ]:
A = algo()

In [ ]:
A.fit(sp_mat)

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
# A.save(f"{os.getcwd()}/B")

In [ ]:
A.load(f"{os.getcwd()}/B.npy")

In [ ]:
metric1 = evaluate.NDCGK(20)

In [ ]:
metric2 = evaluate.RecallK(20)

In [ ]:
evaluator = evaluate.FoldInPercentage(0.8, sp_mat)

In [ ]:
for _in, _out in evaluator:
    
    if _in.nnz == 0 or _out.nnz == 0:
        print("oops")
        continue
        
    
    
    X_pred = A.predict(_in)
    
    metric1.update(X_pred[0], _out)
    metric2.update(X_pred[0], _out)


In [ ]:
metric1.value

In [ ]:
metric2.value

In [ ]:
print(f"NDCG@20 = {metric1.value}, Recall@20 = {metric2.value}")